In [226]:
# !pip install \
#   langchain_community \
#   langchain_pinecone \
#   langchain_openai \
#   unstructured \
#   langchain-text-splitters \
#   pinecone-text \
#   langchain-cohere    

In [227]:
from langchain_pinecone import PineconeVectorStore
from langchain_community.retrievers import (
    PineconeHybridSearchRetriever)
from pinecone.grpc import PineconeGRPC as Pinecone
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.document_loaders import TextLoader
from langchain_core.runnables import Runnable
from pinecone import ServerlessSpec
import pandas as pd
import itertools
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
from dotenv import load_dotenv
import pinecone
import uuid
import os
import glob
import hashlib
from tqdm.autonotebook import tqdm
from langchain.storage import InMemoryStore
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [228]:
#  import nltk
#  nltk.download('punkt')

In [229]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Fetch API keys from environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
pinecone_api_key = os.getenv('P"INECONE_API_KEY')


# Set the environment variables
if openai_api_key:
    os.environ['OPENAI_API_KEY'] = openai_api_key
if pinecone_api_key:
    os.environ['PINECONE_API_KEY'] = pinecone_api_key 

#Verify that the keys are loaded
#print(f"OpenAI API Key: {os.environ.get('OPENAI_API_KEY')}")
#print(f"Pinecone API Key: {os.environ.get('PINECONE_API_KEY')}")

In [230]:
directory = 'data'

def load_docs(directory):
    loader = DirectoryLoader(directory)
    docs = loader.load()
    return docs

docs = load_docs(directory)

In [231]:
docs

[Document(metadata={'source': 'data/Appointments Module ✓.docx'}, page_content="support@vitafyhealth.com | 1-866-4-Vitafy\n\nAppointments Module\n\nSeptember, 2023\n\nOverview\n\nStreamline the process of adding and organizing your clients' appointments. Whether your clients need to schedule multiple appointments for your services or make changes to existing ones, our appointments module allows you to seamlessly add new appointments, check in clients, reschedule, edit, or cancel appointments with ease.\n\nAppointment Scheduling\n\nEffortlessly schedule or reschedule the date and time for your clients' appointments with precision and convenience.\n\nAppointment Check-in\n\nThis module provides a seamless check-in and check-out experience for your clients during their appointment times, ensuring a smooth and efficient process.\n\nHow To\n\nAdd an appointment\n\nFilter appointments\n\nCheck-in / check-out  a client for their appointment\n\nReschedule or cancel an existing appointment\n\nC

In [232]:
print(len(docs))

17


In [233]:
print(docs[0].page_content[0:100])
print(docs[0].metadata)

support@vitafyhealth.com | 1-866-4-Vitafy

Appointments Module

September, 2023

Overview

Streamlin
{'source': 'data/Appointments Module ✓.docx'}


In [234]:

os.environ["HUGGINGFACEHUB_API_TOKEN"]="HUGGINGFACEHUB_API_TOKEN"
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",  #response time is 9s  #infloat/e5-base-V2 has 3.53sec response time.
)
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x322162510>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x15e764740>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [235]:
import getpass
import os
import time
from pinecone import Pinecone, ServerlessSpec

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)
import time

index_name = "test-2"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)
#pc.list_indexes()

In [236]:
chunk_size = 500 
chunk_overlap = 50  

text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

split_docs = text_splitter.split_documents(docs)

In [237]:
# index_name = "test-2"
# #vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
# vectorstore = PineconeVectorStore.from_documents(split_docs, embeddings, index_name=index_name)
# retriever = vectorstore.as_retriever()

## Common ID Prefixes

In [238]:
# import os

# # Generate a sequential document prefix like doc-1, doc-2, etc.
# def generate_prefix(doc_counter):
#     return f"doc-{doc_counter}"

# # Load documents from a directory
# def load_docs(directory):
#     loader = DirectoryLoader(directory)
#     docs = loader.load()
#     if docs:  # Ensure docs are loaded properly
#         return docs
#     else:
#         raise ValueError("No documents were loaded from the directory.")

# def upload_documents_with_prefix(directory, vectorstore, index):
#     docs = load_docs(directory)  

#     if not docs:
#         raise ValueError("No documents found in the directory.")

#     # Add counter for document IDs
#     doc_counter = 1

#     for doc in docs:
#         document_name = os.path.basename(doc.metadata["source"])
#         parent_prefix = generate_prefix(doc_counter)  # Sequential prefix like doc-1, doc-2
        
#         # Print the document name and the generated ID (e.g., id-1, id-2)
#         print(f"Uploading document '{document_name}' with prefix: {parent_prefix} and ID: id-{doc_counter}")

#         split_docs = text_splitter.split_documents([doc])

#         vectors = []
#         for i, chunk in enumerate(split_docs):
#             chunk_id = f"id-{doc_counter}_{parent_prefix}_chunk_{i+1}"  # Add "id-X" and "doc-X" prefix
#             chunk_vector = embeddings.embed_documents([chunk.page_content])[0]

#             # Store the actual page content in metadata instead of chunk number
#             metadata = {
#                 "source": doc.metadata["source"],
#                 "page_content": chunk.page_content  # Store the content of the document
#             }

#             vectors.append({"id": chunk_id, "values": chunk_vector, "metadata": metadata})
        
#         # Upload the vectors to Pinecone
#         index.upsert(vectors=vectors)

#         # Increment the document counter for the next document
#         doc_counter += 1

# directory = 'data'

# # Assuming `vectorstore` and `index` are already initialized
# vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

# # Upload documents with prefix and print ID
# upload_documents_with_prefix(directory, vectorstore, index)


## List of the IDs of all records


In [239]:
# from pinecone.grpc import PineconeGRPC as Pinecone

# index = pc.Index("test-2")
# ids = index.list(namespace='')

# for ids in index.list(namespace=''):
#     print(ids)
    
# #['doc1#v1#chunk1', 'doc1#v1#chunk2', 'doc1#v1#chunk3']

In [240]:
# import os


# from PyPDF2 import PdfReader  # For reading PDF files
# import docx 
# index_name = "test-2"
# directory = 'data'
# # 1. Upload data from 'data' directory
# # Function to read text from different file types
# def read_document(file_path):
#     if file_path.endswith('.txt'):
#         with open(file_path, 'r', encoding='utf-8') as file:
#             return file.read()
#     elif file_path.endswith('.pdf'):
#         with open(file_path, 'rb') as file:
#             reader = PdfReader(file)
#             return "\n".join(page.extract_text() for page in reader.pages if page.extract_text())
#     elif file_path.endswith('.docx'):
#         doc = docx.Document(file_path)
#         return "\n".join(paragraph.text for paragraph in doc.paragraphs)
#     else:
#         return None  # Unsupported file type
        
# # 1. Upload data from 'data' directory
# for filename in os.listdir(directory):
#     file_path = os.path.join(directory, filename)
    
#     # Read the content of the document based on its type
#     document_text = read_document(file_path)
    
#     if document_text:  # Only proceed if the document was read successfully
#         # 2. Generate embedding for the document
#         vector = embeddings.embed_documents([document_text])[0]  # Use embed_documents for document embeddings

        
#         # 3. Generate a unique vector ID (using the filename without extension)
#         #vector_id = os.path.splitext(filename)[0]  # Unique ID based on filename
        
#         vector_id = str(uuid.uuid4())  # Generate a unique UUID

        
#         # Prepare metadata (optional)
#         metadata = {
#             "source": filename,
#             "page_content": document_text[:100]  # Store first 100 characters as a preview (optional)
#         }
        
#         # Upsert the vector into Pinecone
#         index.upsert(vectors=[(vector_id, vector)], metadata=metadata)

#         # 4. Print vector ID and document name
#         print(f"Uploaded Document - Vector ID: {vector_id}, Document Name: {filename}")

# print("All documents uploaded successfully!")


In [241]:
import os
import uuid
from PyPDF2 import PdfReader  # For reading PDF files
import docx


directory = 'data'

# Function to read text from different file types
def read_document(file_path):
    if file_path.endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    elif file_path.endswith('.pdf'):
        with open(file_path, 'rb') as file:
            reader = PdfReader(file)
            return "\n".join(page.extract_text() for page in reader.pages if page.extract_text())
    elif file_path.endswith('.docx'):
        doc = docx.Document(file_path)
        return "\n".join(paragraph.text for paragraph in doc.paragraphs)
    else:
        return None  # Unsupported file type

# Upload documents and store metadata including source and text
def upload_documents(directory, embeddings, index):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        
        # Read the content of the document based on its type
        document_text = read_document(file_path)
        
        if document_text:  # Only proceed if the document was read successfully
            # 1. Generate embedding for the document
            vector = embeddings.embed_documents([document_text])[0]  # Use embed_documents for document embeddings

            # 2. Generate a unique vector ID (UUID)
            vector_id = str(uuid.uuid4())  # Generate a unique UUID
            
            # 3. Prepare metadata (source and content preview)
            metadata = {
                "source": filename,  # The document name as source
                "page_content": document_text[:100]  # Store first 100 characters as a preview of the document
            }
            
            # 4. Upsert the vector into Pinecone with metadata
            #index.upsert(vectors=[(vector_id, vector)], metadata=metadata)
            index.upsert(vectors=[(vector_id, vector, metadata)])  # Make sure to include metadata here

            # 5. Print vector ID and document name
            print(f"Uploaded Document - Vector ID: {vector_id}, Document Name: {filename}")

    print("All documents uploaded successfully!")

# Call the function to upload documents from the directory
upload_documents(directory, embeddings, index)


Uploaded Document - Vector ID: 271d58b9-cbe2-4467-a5b4-0b5d4e0386c2, Document Name: Appointments Module ✓.docx
Uploaded Document - Vector ID: b1703c71-35e0-43a8-bfee-c49116118965, Document Name: Comms Module ✓.docx
Uploaded Document - Vector ID: 4d956c40-7bb1-4fee-9273-72c46a9e9c13, Document Name: Clients - Cases Module ✓.docx
Uploaded Document - Vector ID: 051837e7-85ae-4e96-9560-c59b68487f56, Document Name: Clients Module ✓.docx
Uploaded Document - Vector ID: ca790f90-ce84-408b-8bd9-775d03c4edf2, Document Name: Leads Module ✓.docx
Uploaded Document - Vector ID: 5c37bcb1-0dfd-4858-8461-66909adad8a3, Document Name: Activity Logs Module ✓.docx
Uploaded Document - Vector ID: 7aebcb93-8915-4772-b236-cded60a8ae58, Document Name: Settings Module.docx
Uploaded Document - Vector ID: 2b9091c3-a2ae-4493-adc3-29768431e860, Document Name: Reports Module ✓.docx
Uploaded Document - Vector ID: a4f96c99-86d5-4a89-929a-bd80856e3f31, Document Name: Cases Module ✓.docx
Uploaded Document - Vector ID: 730

In [242]:
# # Document (vector) ID to delete
# document_id = "d07fbd0a-927d-4fe3-8ddc-2769bc0c91e5"

# # Delete the document from the index by its ID
# index.delete(ids=[document_id])

# print(f"Document with ID {document_id} has been deleted.")


In [243]:
keyword = """ forecasting on the future values (any 50 word)"""

In [244]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retrieved_docs = retriever.invoke("""forecasting on the future values (any 50 word) """)
print(format_docs(retrieved_docs))

interrupted by weather or other circumstance. It is generally accepted to be a fair

and accurate method of setting a target score, as it attempts to predict what

of five points in the cumulative ranking and will be ranked accordingly.

of five points in the cumulative ranking and will be ranked accordingly.

interrupted by weather or other circumstance. It is generally accepted to be a fair

and accurate method of setting a target score, as it attempts to predict what


In [245]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=1.0,
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    #chain_type="stuff",
    retriever=vectorstore.as_retriever(),

)
qa.invoke(keyword)

{'query': ' forecasting on the future values (any 50 word)',
 'result': 'Forecasting future values involves analyzing historical data and trends to make informed predictions about future events or performance. This process often uses statistical methods, machine learning algorithms, or expert judgment to anticipate upcoming outcomes, helping businesses and individuals make strategic decisions and plan effectively for the future.'}

In [246]:
# custom_rag_template = PromptTemplate.from_template(template)


# # Create the parallel chain
# My_rag_chain = RunnableParallel(
#     {
#         "context": retriever | format_docs,  # Retriever retrieves context and formats it
#         "question": RunnablePassthrough()    # Pass the question directly
#     }
# ) | custom_rag_template | llm | StrOutputParser()

# # Now pass the keyword (string) directly to the chain
# keyword = "summarize ai paper"
# response = My_rag_chain.invoke(keyword)  # Just pass the keyword, not a dictionary
# print("Response from chain:", response)


In [247]:
# custom_rag_template = PromptTemplate.from_template(template)

# # Create the parallel chain
# My_rag_chain = RunnableParallel(
#     {
#         "context": retriever | format_docs,
#         "question": RunnablePassthrough()
#     }
# ) | custom_rag_template | llm | StrOutputParser()

# ## My chain : Retriever(Pinecone) | custom_rag_template(prompt) | llm | StrOutputParser()

In [248]:
# My_rag_chain.invoke(keyword)
# #print("Chat with your Documents:")
# print(My_rag_chain.invoke(keyword))

In [249]:
# # Specify the vector ID of the document you want to delete
# vector_id_to_delete = "0820821f-2edf-44da-a27e-afb85963b010" 

# # Perform the delete operation
# index.delete(ids=[vector_id_to_delete])

# print(f"Document with Vector ID: {vector_id_to_delete} has been deleted successfully.")

## Metadata Filtering

In [250]:


# # Define the metadata filter for deletion
# metadata_filter = {"source": "/Users/user/Downloads/colab/data/Settings Module.docx"}

# # Delete documents matching the metadata filter
# deleted_count = vectorstore.delete(filter=metadata_filter)

# print(f"Deleted {deleted_count} documents matching the filter.")

## Delete by Common ID Prefixes

In [251]:
# def delete_document_by_id(document_id, index):
#     try:
#         # Delete the document chunks associated with the document_id (e.g., "id-2")
#         index.delete(ids=[document_id])

#         # Print success message
#         print(f"Document with ID '{document_id}' deleted successfully.")
    
#     except Exception as e:
#         print(f"An error occurred while deleting the document: {str(e)}")
        

# delete_document_by_id("id-1", index)


In [252]:
#index.delete(ids=["id-1"])

In [253]:
# for ids in index.list(prefix='doc-1#'):
#     print(ids)

In [254]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retrieved_docs = retriever.invoke("""how to create a membership """)
print(format_docs(retrieved_docs))

Click on ellipses > modify membership

Click +Add on the modify membership popup

Select membership from the list

Select membership from the list

Click on drop down to switch to Public/Private bundle (Private can only be chosen/seen by admin).

Add start date of new membership.

Select Continue.

Confirm.

To add additional membership: 

Go to the clients module from the left menu bar

Find the client you want to update membership for

Click on ellipses > modify membership

Go to the clients module from the left menu bar

Find the client you want to update membership for

Go to ellipses > edit profile

Membership tab

Choose the desired membership type and start date.


In [255]:
qa.invoke("tell me about comms modules")

Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.


{'query': 'tell me about comms modules',
 'result': 'The Comms Module, launched in September 2023, allows efficient monitoring of all incoming and outgoing email communications within the portal. This module ensures seamless communication by promptly notifying clients, administrators, agents, and advocates via email as per their specific needs and roles. Here’s an overview of its main features:\n\n### Inbox and Outbox:\n- **Inbox**: Displays all emails received by users from the portal, such as tagged notes and appointment reminders.\n- **Outbox**: Displays all emails sent from the portal, including activation emails, receipts, event reminders, and tagged notes.\n\n### Consistent Communication:\n- Ensure direct and seamless communication with the ability to track all interactions.\n- Manage email templates conveniently through the settings module to take control of your messaging.\n\n### How to Use:\n- **Browse and Filter Emails**: Use the module to view any notifications or communicat

In [260]:
template = """You are an expert LLM assistant specialized in answering questions based solely on the information provided in the uploaded documents (PDF, DOCX, or TXT formats). Use only the information from the documents to respond accurately and clearly to each question.

Guidelines:
1. Provide concise and informative answers.
2. If the answer is not found in the uploaded documents, state, "The answer is not specifically mentioned in the provided documents."
3. Avoid using outside knowledge or assumptions. Stick strictly to the content in the documents.
4. Maintain a professional and helpful tone thinking you are giving service to the customer for their documents.
5. Answer for normal conversation questions like "Hi", "Hey", "Hello", "How are you?", and many others with the answer: "Hello, How can I assist you?".
6. If the question is on "summarize" or "summarization", then summarize the documents to (1/4)th the size of the original documents.

Question: {question}

Context: {context}

Answer:
"""

prompt = template.format(question=keyword, context=format_docs(retrieved_docs))

# Create the LLM
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=1.0,
)

# Set up the retrieval QA with custom response for missing answers
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True  # Ensures that source documents are returned
)

# Wrapper function to handle missing answers
def get_answer(keyword):
    response = qa.invoke(keyword)
    
    # Check if the response contains valid data
    result = response.get('result', '')
    if "The answer is not specifically mentioned" in result or not result.strip():
        return "Precise answer not found in documents, try another prompt."
    else:
        return result

# Test the response with the keyword query

final_answer = get_answer("7. What is understood by an Electoral College?")
print(final_answer)

An Electoral College is a group of persons elected through general voters for the purpose of electing the President, Vice-President, or other individuals holding important positions. In Nepal, the Constitution has made provisions for the election of the President, Vice-President, and members of the National Assembly to be conducted through an Electoral College.


In [257]:
pc.describe_index("test-2")


{
    "name": "test-2",
    "dimension": 1536,
    "metric": "dotproduct",
    "host": "test-2-5vwf04k.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "deletion_protection": "disabled"
}

In [259]:
# Document (vector) ID to delete
document_id = "3db7c16b-a763-429b-bcc0-c0ece03dccc5"

# Delete the document from the index by its ID
index.delete(ids=[document_id])

print(f"Document with ID {document_id} has been deleted.")


Document with ID 3db7c16b-a763-429b-bcc0-c0ece03dccc5 has been deleted.
